In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
each_words=[]
batches=[]
files=[]
ignore=["?","!"]
data_file=open("D:/deva/intents-medical.json").read()
data=json.loads(data_file)
data

{'intents': [{'tag': 'Cuts',
   'patterns': ['What to do if Cuts?',
    'How to cure Cuts?',
    'Which medicine to apply for Cuts?',
    'what to apply on cuts?',
    'Cuts'],
   'responses': ['Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.'],
   'context_set': ''},
  {'tag': 'Abrasions',
   'patterns': ['how do you treat abrasions?',
    'Do Abrasions cause scars?',
    'Abrasions',
    'what to do if abrasions?',
    'Which medicine to apply for abrasions?',
    'How to cure abrasions?'],
   'responses': ['Begin with washed hands.Gently clean the area with cool to lukewarm water and mild soap. Remove dirt or other particles from the wound using sterilized tweezers.For a mild scrape thatâ€™s not bleeding, leave the wound uncovered.If t

In [12]:
for content in data["intents"]:
    for content1 in content["patterns"]:
        w=nltk.word_tokenize(content1)
        each_words.extend(w)
        files.append((w,content["tag"]))
        if content["tag"] not in batches:
            batches.append(content["tag"])

In [13]:
each_words=[lemmatizer.lemmatize(w.lower()) for w in each_words if w not in ignore]
each_words=sorted(list(set(each_words)))
batches=sorted(list(set(batches)))
print(len(files),"files")
print(len(batches),"batches",batches)
print(len(each_words),"each_words are lemmatized",each_words)
pickle.dump(each_words,open("each_words.pkl","wb"))
pickle.dump(batches,open("batches.pkl","wb"))

188 files
44 batches ['Abdonominal Pain', 'Abrasions', 'Broken Toe', 'Bruises', 'CPR', 'Chemical Burn', 'Choking', 'Cold', 'Cough', 'Cuts', 'Diarrhea', 'Drowning', 'Eye Injury', 'Fainting', 'Fever', 'Fracture', 'Frost bite', 'Gastrointestinal problems', 'Head Injury', 'Headache', 'Heat Exhaustion', 'Heat Stroke', 'Insect Bites', 'Nasal Congestion', 'Normal Bleeding', 'Poison', 'Pulled Muscle', 'Rash', 'Rectal bleeding', 'Skin problems', 'Sore Throat', 'Splinter', 'Sprains', 'Strains', 'Sun Burn', 'Teeth', 'Testicle Pain', 'Vertigo', 'Wound', 'animal bite', 'nose bleed', 'seizure', 'snake bite', 'stings']
124 each_words are lemmatized ['a', 'abdonominal', 'abrasion', 'allergy', 'am', 'an', 'animal', 'apply', 'better', 'bit', 'bite', 'bitten', 'bleed', 'bleeding', 'blocked', 'bring', 'broken', 'bruise', 'burn', 'by', 'cause', 'chemical', 'choke', 'choked', 'choking', 'cold', 'congestion', 'cough', 'cpr', 'cream', 'cure', 'cut', 'diagnose', 'diarrhea', 'do', 'doe', 'dog', 'drowned', 'drow

In [14]:
training_data=[]
box_empty=[0]*len(batches)
for file1 in files:
    bag=[]
    pat_words=file1[0]
    pat_words=[lemmatizer.lemmatize(word.lower()) for word in pat_words]
    for w in each_words:
        bag.append(1) if w in pat_words else bag.append(0)
    box_row=list(box_empty)
    box_row[batches.index(file1[1])]=1
    training_data.append([bag,box_row])
random.shuffle(training_data)
training_data=np.array(training_data)
x_train=list(training_data[:,0])
y_train=list(training_data[:,1])
print("Training Data Created")

Training Data Created


In [15]:
Machine=Sequential()
Machine.add(Dense(244,input_shape=(len(x_train[0]),),activation="relu"))
Machine.add(Dropout(0.5))
Machine.add(Dense(128,activation="relu"))
Machine.add(Dropout(0.5))
Machine.add(Dense(len(y_train[0]),activation="softmax"))
gradiant=SGD(momentum=0.9,nesterov=True,lr=0.01,decay=1e-6)
Machine.compile(loss="categorical_crossentropy",optimizer=gradiant,metrics=["accuracy"])
hist=Machine.fit(np.array(x_train),np.array(y_train),epochs=499,batch_size=5,verbose=1)
Machine.save("chatbot machine.h5",hist)
print('Machine Created')

Epoch 1/499
38/38 [==============================] - 0s 2ms/step - loss: 3.8469 - accuracy: 0.0213
Epoch 2/499
38/38 [==============================] - 0s 1ms/step - loss: 3.7666 - accuracy: 0.0426
Epoch 3/499
38/38 [==============================] - 0s 821us/step - loss: 3.6922 - accuracy: 0.0691
Epoch 4/499
38/38 [==============================] - 0s 821us/step - loss: 3.6380 - accuracy: 0.1011
Epoch 5/499
38/38 [==============================] - 0s 1ms/step - loss: 3.5348 - accuracy: 0.1277
Epoch 6/499
38/38 [==============================] - 0s 1ms/step - loss: 3.4869 - accuracy: 0.1543
Epoch 7/499
38/38 [==============================] - 0s 987us/step - loss: 3.3022 - accuracy: 0.1596
Epoch 8/499
38/38 [==============================] - 0s 887us/step - loss: 3.0920 - accuracy: 0.2660
Epoch 9/499
38/38 [==============================] - 0s 821us/step - loss: 2.8654 - accuracy: 0.2713
Epoch 10/499
38/38 [==============================] - 0s 1ms/step - loss: 2.5864 - accuracy: 0.3298

In [16]:
from keras.models import load_model
Machine=load_model("chatbot machine.h5")
data=json.loads(open("D:/deva/intents-medical.json").read())
each_words=pickle.load(open("each_words.pkl","rb"))
batches=pickle.load(open("batches.pkl","rb"))

In [17]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
def bow(sentence,each_words,show_details=True):
    sentence_words=clean_up_sentence(sentence)
    bag=[0]*len(each_words)
    for s in sentence_words:
        for i,w in enumerate(each_words):
            if w==s:
                bag[i]=1
                if show_details:
                    print("found in bag:%s" %w)
    return(np.array(bag))   
def predict_classes(sentence,Machine):
    p=bow(sentence,each_words,show_details=False)
    res=Machine.predict(np.array([p]))[0]
    ERROR_THRESHOLD=0.25
    results =[[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x:x[1],reverse=True)
    return_list=[]
    for r in results:
        return_list.append({"intent":batches[r[0]],"probablity":str(r[1])})
    return return_list
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_classes(msg, Machine)
    res = getResponse(ints, data)
    return res    
    

In [18]:
chatbot_response("fever")

'To treat a fever at home: 1)Drink plenty of fluids to stay hydrated. 2)Dress in lightweight clothing. 3)Use a light blanket if you feel chilled, until the chills end. 4)Take acetaminophen (Tylenol, others) or ibuprofen (Advil, Motrin IB, others). 5) Get medical help if the fever lasts more than five days in a row.'

In [19]:
import tkinter
from tkinter import *
def send():
    Messege = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if Messege != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + Messege + '\n\n')
        ChatLog.config(foreground="white", font=("Calibri", 12 ))
        Result = chatbot_response(Messege)
        ChatLog.insert(END, "Bot: " + Result + '\n\n')
        ChatLog.config(foreground="white", font=("Calibri", 12,'bold' ))
        ChatLog.yview(END)
base = Tk()
base.title("Chat With Robot")
base.geometry("600x600")
base.resizable(width=FALSE, height=FALSE)
ChatLog = Text(base, bd=0, bg="#87ceeb", height="8", width="50", font="Calibri",)
ChatLog.config(state=DISABLED)
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="arrow")
ChatLog['yscrollcommand'] = scrollbar.set
SendButton = Button(base, font=("Calibri",15,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Calibri")
scrollbar.place(x=576,y=6, height=586)
ChatLog.place(x=6,y=6, height=586, width=570)
EntryBox.place(x=6, y=551, height=51, width=443)
SendButton.place(x=450, y=551, height=50)
base.mainloop()